# COMP9417 Homework 1

## Preprocessing

In [47]:
import pandas as pd
import numpy as np

In [48]:
df = pd.read_csv("Advertising.csv")

In [49]:
def normalise(vector):
    vector = [(x - min(vector))/(max(vector)-min(vector)) for x in vector]
    return vector

In [55]:
# normalise the data
normalised = df[['TV', 'Radio', 'Newspaper']].apply(normalise)

In [56]:
normalised['Sales'] = df['Sales'] 

## Segment Training and Test Data

In [57]:
training_set = normalised[0:190]
test_set = normalised[190:200]

## Gradient Descent

In [ ]:
class Fit:
    
    def __init